In [1]:
import torch
import torch.nn as nn

/workspaces/data-mining/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [4]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [5]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [14]:
class TransformerBlock( nn.Module ):
    def __init__( self, cfg ):
        super().__init__()
        self.norm1 = LayerNorm( cfg["emb_dim"] )
        self.att = MultiHeadAttention( 
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"] 
        )
        self.dropout = nn.Dropout( cfg["drop_rate"] )
        self.norm2 = LayerNorm( cfg["emb_dim"] )
        self.feedforward = FeedForward( cfg )
        
    def forward( self, x ):
        
        # the attention chunk in this transformer:
        shortcut = x # save input for the skip connection
        x = self.norm1( x ) # normalize input
        x = self.att( x ) # Shape [batch_size, num_tokens, emb_size]
        x = self.dropout( x ) # dropout to prevent overfitting, d rate = 0 means no dropout, d rate = 1 means drop all
        x = x + shortcut # add the original input back
        
        # the feed forward chunk:
        shortcut = x
        x = self.norm2( x ) 
        x = self.feedforward( x )
        x = self.dropout( x )
        x = x + shortcut # add the original input back
        
        return x

In [15]:
# instantiate TransformerBlock
block = TransformerBlock( GPT_CONFIG_124M )

In [16]:
input = torch.randn( 3, 4, 768 )

In [17]:
input

tensor([[[-0.0077,  0.8219,  0.5323,  ..., -1.1732,  0.9125,  2.3440],
         [-0.3028,  0.7909,  1.2056,  ..., -1.0923,  0.4948, -0.4412],
         [-0.0275,  0.8281,  1.6134,  ...,  1.7769, -0.5143, -0.2235],
         [-0.2123, -0.8729, -0.6700,  ...,  1.8271, -0.1797, -0.5325]],

        [[ 0.5325,  0.2909,  0.2519,  ...,  0.7424,  0.4273, -0.2163],
         [-0.5867,  0.6235, -0.9185,  ..., -1.8476,  0.4272,  1.2885],
         [-0.3963,  0.2336,  0.0529,  ..., -1.0745, -1.2487,  0.2577],
         [-1.1621,  0.4648,  1.4962,  ...,  0.5376, -0.1340,  0.0946]],

        [[-0.4547,  0.2888, -0.2243,  ...,  2.2969, -1.0171, -0.8869],
         [-0.5201, -1.2108, -0.1405,  ..., -1.0286,  1.3420, -0.3385],
         [-0.1084,  0.5241, -0.1635,  ...,  0.6100, -1.0456,  0.0900],
         [ 1.2121,  0.7435, -1.1256,  ..., -0.4307, -1.0787,  0.2056]]])

In [18]:
output = block( input )

In [19]:
output.shape

torch.Size([3, 4, 768])

In [21]:
output

tensor([[[ 7.9515e-02,  1.1896e+00,  3.1306e-01,  ..., -8.1049e-01,
           3.1943e-01,  2.5390e+00],
         [ 2.4804e-02,  1.6895e+00,  8.2603e-01,  ..., -1.0687e+00,
           5.3111e-01, -1.8523e-01],
         [ 1.7334e-01,  1.2947e+00,  1.5817e+00,  ...,  2.1402e+00,
          -7.5819e-01, -8.1502e-02],
         [-3.5495e-01, -8.0119e-01, -7.9000e-01,  ...,  1.7943e+00,
          -1.2406e-01, -5.3619e-01]],

        [[ 2.9556e-01,  4.6027e-01, -1.0702e-01,  ...,  6.3985e-01,
           1.1908e+00, -3.6670e-01],
         [-3.0337e-01,  6.3331e-01, -8.6674e-01,  ..., -1.8407e+00,
           6.9811e-01,  1.0158e+00],
         [-1.9206e-01,  8.7416e-01,  1.1439e-01,  ..., -8.3446e-01,
          -1.4393e+00,  4.6376e-01],
         [-8.6883e-01,  1.0713e+00,  1.2312e+00,  ...,  6.4941e-01,
          -1.1127e-01,  8.1151e-02]],

        [[-1.4278e-01,  6.3987e-01, -2.7495e-03,  ...,  3.1640e+00,
          -1.8440e+00, -6.7558e-01],
         [-3.9942e-01, -1.1929e+00, -3.8625e-01,  .